In [4]:
!pip install gender-guesser

In [13]:
import gender_guesser.detector as gender
d = gender.Detector()
d.get_gender(u"Igal")


'male'

In [14]:
from pathlib import Path
import requests
from lxml import etree
from tqdm import tqdm
from collections import Counter
import pandas as pd
from typing import List
from dataclasses import dataclass, field

ModuleNotFoundError: No module named 'lxml'

In [ ]:
@dataclass
class Item:
    id: int = None
    author: str = None
    journal_title: str = None
    book_title: str = None
    book_chapter: str = None
    article_title: str = None
    year: str =None
    file: str = None
    collection: str = None
    jstor_url: str = None
    ngrams: List[tuple] = field(default_factory=list)
    
    #list of all tokens 
    def tokens(self):
        tokens = [a[0] for a in self.ngrams]
        return tokens
        
    #frequency for a specific token
    def token_freq(self, token:str):
        a = [a for a in self.ngrams if a[0] == token]
        return int(a[0][1])

    #frequencies for a list of token
    def token_list(self, tokens:list):
        return [a for a in self.ngrams if a[0] in tokens]
           
    def hatebase_terms(self):
        intersection = set([a[0].lower() for a in self.ngrams]).intersection(set(hatebase))
        return [a for a in self.ngrams if a[0].lower() in intersection] 
        
     

In [ ]:
def make_items(path):
    current_dir = Path(path)
    return_items = []
    if (current_dir / 'metadata').exists():
        count = 0 
        for file in tqdm((current_dir / 'metadata').iterdir(), 
                         total=len(list((current_dir / 'metadata').iterdir()))):
            item = Item()
            item.id = count
            count += 1
            item.file = str(file.name)
            item.collection = str(current_dir).split('/')[-1]
            
            
            root = etree.fromstring(file.read_bytes())
            try:
                item.jstor_url = root.xpath('//self-uri')[0].attrib['{http://www.w3.org/1999/xlink}href']
            except IndexError:
                continue
            if 'article' in file.name:
                try:
                    item.journal_title = root.xpath('//journal-title')[0].text
                except IndexError:
                    continue
                if root.xpath('//copyright-year'):
                    item.year = root.xpath('//copyright-year')[0].text
                if root.xpath('//string-name'):
                    item.author = root.xpath('//string-name')[0].text #only first author
                if root.xpath('//article-title'): #TODO not working, no results
                    article_title = root.xpath('//article-title')[0].text 
            if 'book' in file.name:
                if root.xpath('//book-title'):
                    item.book_title = root.xpath('//book-title')[0].text
                book_part_id = file.stem.split('_')[-1]
                i = int(book_part_id.split('.')[-1])
                try:
                    item.book_chapter = root.xpath('//title')[i].text   
                except IndexError:
                    pass
            
            sub_dirs = ['ngram1','ngram2','ngram3']
            for sub_dir in sub_dirs:
                #book-chapter-10.2307_j.ctt1pc5dgp.4.xml maps to 
                #book-chapter-10.2307_j.ctt1pc5dgp.4-ngram1.txt
                txt_file = str(file.stem)+'-'+sub_dir+'.txt'
                ngram_file = (current_dir / sub_dir / txt_file )
                if ngram_file.exists():
                    text = ngram_file.read_text()
                    for line in text.split('\n'):
                        word=line.split('\t')[0]
                        freq=line.split('\t')[-1]
                        item.ngrams.append((word,freq))
            return_items.append(item)
            
        return return_items
                
                
    
    else:
        print('Missing metadata directory')
        raise SystemExit(0)
        
    
directories = ['/home/ajanco/projects/slavic_review/slavic_review_data/SlavicStudiesCluster1991to2020'] #,'/home/ajanco/projects/slavic_review/slavic_review_data/AfricanAmericanStudiesCluster1985to2020']
main_items = []
for directory in directories:
    main_items.extend(make_items(directory))

    